# Imports

In [17]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score, r2_score


df = pd.read_csv('./mdata/df_merged_optimized.csv')

In [18]:
df_1 = df.copy()
df_1['StateHoliday_binary'] = df.StateHoliday.map(lambda x: 1 if x == 0 else 0)
df_1.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,StateHoliday_binary
0,2013-01-01,1115,2,0,0,False,False,a,True,d,c,5350,0,0,True,22,2012,m,0
1,2013-01-02,1115,3,3697,305,True,False,d,True,d,c,5350,0,0,True,22,2012,m,0
2,2013-01-03,1115,4,4297,300,True,False,d,True,d,c,5350,0,0,True,22,2012,m,0
3,2013-01-04,1115,5,4540,326,True,False,d,True,d,c,5350,0,0,True,22,2012,m,0
4,2013-01-05,1115,6,4771,339,True,False,d,True,d,c,5350,0,0,True,22,2012,m,0


In [21]:
# feature engineer a promo running time
from datetime import datetime
year = datetime.today().year
df_1['Promo2_time'] = year - df.Promo2SinceYear
df_1.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,StateHoliday_binary,Promo2_time
0,2013-01-01,1115,2,0,0,False,False,a,True,d,c,5350,0,0,True,22,2012,m,0,11
1,2013-01-02,1115,3,3697,305,True,False,d,True,d,c,5350,0,0,True,22,2012,m,0,11
2,2013-01-03,1115,4,4297,300,True,False,d,True,d,c,5350,0,0,True,22,2012,m,0,11
3,2013-01-04,1115,5,4540,326,True,False,d,True,d,c,5350,0,0,True,22,2012,m,0,11
4,2013-01-05,1115,6,4771,339,True,False,d,True,d,c,5350,0,0,True,22,2012,m,0,11


In [23]:
#one hot encoding for Foreign/Local Used column
import category_encoders as ce

# create an object of the OneHotEncoder
ce_one = ce.OneHotEncoder(cols=['Promo2','SchoolHoliday','Open']) 
df_oh = ce_one.fit_transform(df_1)

In [45]:
import numpy as np

def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

# Simple Linear RegressionTree

In [55]:
df_oh.head()

,Date,Store,DayOfWeek,Sales,Customers,Open_1,Open_2,Promo,StateHoliday,SchoolHoliday_1,...,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2_1,Promo2_2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,StateHoliday_binary,Promo2_time
0,2013-01-01,1115,2,0,0,1,0,False,a,1,...,5350,0,0,1,0,22,2012,m,0,11
1,2013-01-02,1115,3,3697,305,0,1,False,d,1,...,5350,0,0,1,0,22,2012,m,0,11
2,2013-01-03,1115,4,4297,300,0,1,False,d,1,...,5350,0,0,1,0,22,2012,m,0,11
3,2013-01-04,1115,5,4540,326,0,1,False,d,1,...,5350,0,0,1,0,22,2012,m,0,11
4,2013-01-05,1115,6,4771,339,0,1,False,d,1,...,5350,0,0,1,0,22,2012,m,0,11


In [58]:
# split the df into X and y
y = df_oh.Sales
cols_to_drop = ['Sales', 'Date', 'Store', 'Promo2SinceYear',
                'StateHoliday', 'Promo', 'Assortment', 'StoreType', 'PromoInterval']

X = df_oh.drop(columns=cols_to_drop, axis=1)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42 )

In [64]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the DecisionTreeRegressor with default parameters
baseline_regressor = DecisionTreeRegressor(random_state=0)

# Train the baseline model on the training data
baseline_regressor.fit(X_train, y_train)

# Predict on the test data
y_pred = baseline_regressor.predict(X_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# rmspe = metric(y_test.values, y_pred)

print("Baseline Model - Mean Squared Error:", mse)
print("Baseline Model - R-squared:", r2)
# print("Baseline Model - Root Mean Squared Percentage Error:", rmspe)

Baseline Model - Mean Squared Error: 563432.0730209395
Baseline Model - R-squared: 0.9611954707510938
Baseline Model - Root Mean Squared Percentage Error: nan


/tmp/ipykernel_80091/1955404748.py:7: RuntimeWarning: divide by zero encountered in true_divide
  return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])
/tmp/ipykernel_80091/1955404748.py:7: RuntimeWarning: invalid value encountered in true_divide
  return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])


In [63]:
y_test.values

array([    0,  3459,  6408, ..., 10015,  5940,  8600])

# Regression Tree

In [41]:
# split the df into X and y
y = df_oh.Sales
cols_to_drop = ['Sales', 'Date', 'Store', 'StateHoliday', 'Promo', 'Assortment', 'StoreType', 'PromoInterval']
X = df_oh.drop(columns=cols_to_drop, axis=1)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42 )

In [43]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV


# Define the parameter grid to search through
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# # Initialize model
regressor = DecisionTreeRegressor(criterion='squared_error', random_state=42)

# Create the GridSearchCV object
grid_search = GridSearchCV(regressor, param_grid, cv=5, n_jobs=-1)

# Fit the model with the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 10}
Best Score: 0.9664259788568466


In [54]:
best_params

{'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 10}

In [44]:
# Train the model with the best parameters
optimized_regressor = DecisionTreeRegressor(random_state=0, **best_params)
optimized_regressor.fit(X_train, y_train)

# Predict using the optimized model
y_pred = optimized_regressor.predict(X_test)

# Calculate evaluation metrics
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 456192.50660649064
R-squared: 0.9685812428624639


# Linear Regression

In [32]:
# split the df into X and y
y = df_oh.Sales
cols_to_drop = ['Sales', 'Date', 'Store', 'StateHoliday', 'Promo', 'Assortment', 'StoreType', 'PromoInterval']
X = df_oh.drop(columns=cols_to_drop, axis=1)

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42 )

In [33]:
from sklearn.linear_model import LinearRegression   
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)

0.8385187184415295

In [35]:
y_pred = reg.predict(X_test)
reg.score(X_test, y_pred)

1.0